In [ ]:
import pandas as pd
import numpy as np
import pyclustertend 
import random
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.preprocessing

### Pregunta 1

In [ ]:
movies = pd.read_csv('movies.csv', encoding = "ISO-8859-1")
variables = movies.select_dtypes(include=np.number)

normalized_variables =(variables-variables.min())/(variables.max()-variables.min())
print(normalized_variables)

### Pregunta 2

In [ ]:
random.seed(123)
X_scale=sklearn.preprocessing.scale(normalized_variables)

pyclustertend.hopkins(normalized_variables, len(normalized_variables))

El valor de hopkins es de 0.03, por lo que vale la pena hacer el agrupamiento.

In [ ]:
pyclustertend.vat(X_scale[:700])